In [ ]:
import pandas as pd
import numpy as np
from math import exp, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [ ]:
df.head()

In [ ]:
print('missing value percentage')
for col in df.columns:
    missing_percent = df[col].isna().sum() / df[col].shape[0] * 100
    if missing_percent > 0:
        print(f'{col:<25} {missing_percent:.2f}%')

In [ ]:
#shameless, i am just learning
df = df.dropna()

In [ ]:
for col in df.select_dtypes(include='object').columns:
    print('columns = ' + col)
    print('unique = ' + f'{list(df[col].unique())}')
    print('')

In [ ]:
df = pd.concat([df, pd.get_dummies(df['gender'], prefix='gender')], axis=1)
df = pd.concat([df, pd.get_dummies(df['major_discipline'], prefix='major')], axis=1)
df = pd.concat([df, pd.get_dummies(df['company_type'], prefix='company')], axis=1)

In [ ]:
cat_rel_exp = ['No relevent experience', 'Has relevent experience']
cat_enrolled = ['no_enrollment', 'Part time course', 'Full time course']
cat_edu_level = ['Graduate', 'Masters', 'Phd']
cat_experience = ['<1'] + [str(i) for i in sorted([int(i) for i in df['experience'].unique() if i not in ['<1', '>20']])] + ['>20']
cat_comp_size = ['<10', '10/49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+']
cat_last_new_job = ['never', '1', '2', '3', '4', '>4']

In [ ]:
oe = OrdinalEncoder(categories=[cat_rel_exp, cat_enrolled, cat_edu_level, cat_experience, cat_comp_size, cat_last_new_job])
df[['enc_rel_exp', 'enc_enroll', 'enc_edu', 'enc_exp', 'enc_com_size', 'enc_last_new_job']] = oe.fit_transform(df[['relevent_experience', 'enrolled_university', 'education_level', 'experience', 'company_size', 'last_new_job']])

In [ ]:
df.head()

In [ ]:
x = df.select_dtypes(include='number').drop(columns=['enrollee_id', 'target'])
y = df['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)

In [ ]:
x_train.shape

In [ ]:
def predict(row, coefficient):
    theta = coefficient[0]
    for i in range(len(row) - 1):
        theta += coefficient[i + 1] * row[i]
    yhat = 1 / (1 + exp(-theta))
    return yhat

In [ ]:
coef = [0] * 23

In [ ]:
for i in range(10):
    yhat = predict(x.iloc[i].values, coef)
    print(f'expected: {y.iloc[i]}, got: {yhat}')

In [ ]:
def grad_descent(x_train, y_train, learning_rate, n_epochs):
    x_train = x_train.values
    y_train = y_train.values
    coef = [0] * len(x_train[0])
    for epoch in range(n_epochs):
        error_sum = 0
        n = 0
        for x, y in zip(x_train, y_train):
            yhat = predict(x, coef)
            error = y - yhat
            error_sum += error ** 2
            coef[0] += learning_rate * error * yhat * (1 - yhat)
            for i in range(len(x) - 1):
                coef[i + 1] += learning_rate * error * yhat * (1 - yhat) * x[i]
        print(f'epoch: {epoch}, error: {error_sum}')
    return coef

In [ ]:
'''def grad_descent2(x_train, y_train, learning_rate, n_epochs):
    x_train = x_train.values
    y_train = y_train.values
    coef = [0] * len(x_train[0])
    for epoch in range(n_epochs):
        error_sum = 0
        n = 0
        for x, y in zip(x_train, y_train):
            yhat = predict(x, coef)
            error = y - yhat
            error_sum += error ** 2
            coef[0] -= learning_rate * 2 * error
            for i in range(len(x) - 1):
                coef[i + 1] -= learning_rate * 2 * error * x[i]
        print(f'epoch: {epoch}, error: {error_sum}')
    return coef'''

In [ ]:
grad_descent(x_train, y_train, 0.05, 10)

In [ ]:
def logistic_regression(x_train, y_train, x_test, learning_rate, n_epochs):
    x_test = x_test.values
    coef = grad_descent(x_train, y_train, learning_rate, n_epochs)
    yhat = list()
    for x in x_test:
        pred = predict(x, coef)
        #print(pred)
        pred = round(pred)
        yhat.append(pred)
    return yhat

In [ ]:
def accuracy(yhat, y_test):
    correct_count = 0
    for i, j in zip(yhat, y_test):
        if i == j:
            correct_count += 1
    acc = correct_count / len(yhat) * 100
    return acc

In [ ]:
yhat = logistic_regression(x_train, y_train, x_test, 0.001, 1908)

In [ ]:
accuracy(yhat, y_test)